In [10]:
######## Task 3 ########
# a) 

# parameters
T = 1
K = 100
S0 = 100
r = 0.02
sigma = 0.2

# a and b 
a = (r - 0.5 * sigma**2) * T / 2
b = sigma * np.sqrt(T / 2)

# payoff function F(x,y)
def F(x, y):
    ST2 = S0 * np.exp(a + b*x)
    ST  = S0 * np.exp(2*a + b*(x + y))
    return max(0.5*(ST2 + ST) - K, 0)

# density g(x,y)
def g(x, y):
    return (1/(2*np.pi)) * np.exp(-0.5*(x**2 + y**2))

# integration grid
N = 200
lim = 5           # integration from -5 to 5 covers most mass of normal
xs = np.linspace(-lim, lim, N)
ys = np.linspace(-lim, lim, N)
dx = xs[1] - xs[0]
dy = ys[1] - ys[0]

# numerical double integral
total = 0
for x in xs:
    for y in ys:
        total += F(x, y) * g(x, y) * dx * dy

# discounting
C = np.exp(-r*T) * total
print("Asian option price:", C)

Asian option price: 6.996284079189083


In [11]:
def norm_cdf(x):
    """Standard normal CDF."""
    return 0.5 * (1.0 + erf(x / sqrt(2.0)))


def european_call_bs(S, K, r, sigma, tau):
    """Black–Scholes price of a European call option."""
    if tau <= 0:
        return max(S - K, 0.0)
    if S <= 0:
        return 0.0
    if K <= 0:
        # option always in the money, price behaves like forward S - K e^{-r tau}
        return S - K * exp(-r * tau)

    d1 = (log(S / K) + (r + 0.5 * sigma**2) * tau) / (sigma * sqrt(tau))
    d2 = d1 - sigma * sqrt(tau)
    return S * norm_cdf(d1) - K * exp(-r * tau) * norm_cdf(d2)


# finite difference method
def asian_fd(
    S0 = 100,
    K = 100,
    r = 0.02,
    sigma = 0.2,
    T = 1,
    M = 400,          # number of spatial (S) steps
    Smax_mult = 4.0   # S_max = Smax_mult * S0
):
    """
    Finite difference scheme for an Asian call:

        payoff = max(0.5*(S_{T/2} + S_T) - K, 0)

    Using the representation at t = T/2:
        V(S, T/2) = 0.5 * Call_BS(S, K_new = 2K - S, tau = T/2)

    We then solve the Black–Scholes PDE backward in time on [0, T/2].
    Returns (price at S0, S-grid, V at t=0).
    """

    # Time split
    T_mid = T / 2       # we solve PDE from T_mid back to 0
    tau = T / 2         # maturity of the inner European call (from T/2 to T)

    # ---- S grid ----
    Smax = Smax_mult * S0
    dS = Smax / M
    S = np.linspace(0, Smax, M + 1)   # S_0, ..., S_M
    N = int(np.ceil(T_mid / 0.000001))
    dt = T_mid / N  

    # ---- terminal condition at t = T/2 ----
    V = np.zeros(M + 1)
    for j, Sj in enumerate(S):
        K_new = 2 * K - Sj
        call_inner = european_call_bs(Sj, K_new, r, sigma, tau)
        V[j] = 0.5 * call_inner

    # Boundary condition at t = T/2:
    # S = 0: option value ~ 0
    V[0] = 0.0

    # ---- precompute α, β, γ coefficients (spatial operator L) ----
    # PDE: dV/dt + 0.5 * sigma ^2  * S^2 * V_SS + r * S * V_S - r * V = 0
    # => dV/dt = -[ 0.5 * sigma ^2  * S^2 * V_SS + r * S * V_S - r * V ] = L[V]
    i = np.arange(1, M)      # interior indices 1..M-1
    Si = i * dS

    # finite-difference approximation of L[V] at S_i:
    # L[V]_i approx = alpha_i V_{i-1} + beta_i V_i + gamma_i V_{i+1} (it is just a reparametrization after grouping the terms of finit difference approximation of V_S, V_SS)
    alpha = 0.5 * sigma**2 * (Si**2) / dS**2 - 0.5 * r * Si / dS
    beta  = -sigma**2 * (Si**2) / dS**2 - r
    gamma = 0.5 * sigma**2 * (Si**2) / dS**2 + 0.5 * r * Si / dS

    # explicit time stepping uses:
    # V_i^{n} = V_i^{n+1} + alpha_i V_{i-1}^{n+1} * dt + beta_i V_i^{n+1} * dt+ gamma_i V_{i+1}^{n+1} * dt
    a = dt * alpha
    b = 1.0 + dt * beta
    c = dt * gamma

    # ---- time stepping: from t = T/2 down to t = 0 ----
    for n in range(N):
        t = T_mid - n * dt
        t_prev = t - dt

        # update boundary conditions at new time t_prev
        V_left = 0.0

        V_new = np.zeros_like(V)
        V_new[0] = V_left

        # interior nodes: vectorized explicit update
        V_im1 = V[0:M-1]   # V_{i-1}^{n+1}
        V_i   = V[1:M]     # V_i^{n+1}
        V_ip1 = V[2:M+1]   # V_{i+1}^{n+1}

        V_new[1:M] = a * V_im1 + b * V_i + c * V_ip1

        V = V_new
        # interpolate V(S0, t=0)
    price_0 = np.interp(S0, S, V)
    return price_0, S, V


price, S_grid, V0 = asian_fd()
print("Asian option price (FD):", price)

Asian option price (FD): 6.995465849077409


In [12]:
# parameters
S0 = 100
K = 100
r = 0.02
sigma = 0.2
T = 1

# Monte Carlo settings
M = 10**5
N = 200  # time steps
dt = T/N

# simulate
S_paths = np.zeros((M, 2))
S = np.full(M, S0)

for i in range(N):
    Z = np.random.randn(M)
    S = S * np.exp((r - 0.5*sigma**2)*dt + sigma*sqrt(dt)*Z)
    if i == N//2:
        S_paths[:,0] = S  # ST/2

S_paths[:,1] = S        # ST

# payoff
payoff = np.maximum(0.5*(S_paths[:,0] + S_paths[:,1]) - K, 0)

# discounted price
price_est = exp(-r*T) * np.mean(payoff)

# confidence interval
std_est = np.std(payoff) / np.sqrt(M)
ci_low = price_est - 1.96 * exp(-r*T) * std_est
ci_high = price_est + 1.96 * exp(-r*T) * std_est

print("Monte-Carlo price =", price_est)
print("95% CI =", (ci_low, ci_high))

Monte-Carlo price = 6.96259335714468
95% CI = (np.float64(6.896800213285454), np.float64(7.028386501003906))


**Applicability when the underlying is not GBM and has no closed-form solution**


Numerical integration method

This method becomes essentially infeasible. The key reason is that the joint distribution of the sampled underlying prices is no longer known explicitly, so the density function needed for integration cannot be derived in closed form. Even if numerical approximations of the transition densities are constructed, the dimensionality becomes too high and the approximation errors accumulate.

Finite-difference method

It could still be applied, but it requires solving the associated pricing PDE, which may no longer be available in closed form or may become higher-dimensional due to path dependence. It also only works because payoff at T/2 can be expressed as European option price analytically via Black–Scholes. For non-GBM or more sampling times, the “option at intermediate time” is not a simple European call, so the method doesn’t directly apply.

Monte Carlo method

Monte Carlo remains applicable with minimal modification. Any SDE can be simulated numerically (e.g., Euler–Maruyama scheme), and the payoff can be approximated from simulated paths. Thus, this is the most flexible and robust method when no analytical solution is available.

**When GBM holds but the number of sampling times is ≥10**

Numerical integration method

The method becomes effectively unusable. With 10 sampling points, the payoff depends on 10 correlated lognormal variables, making the integral 10-dimensional. Numerical quadrature in high dimensions is exponentially expensive, so the method does not scale.

Finite-difference method

The PDE representation becomes high-dimensional because the payoff depends on the running average (or discrete sum). For more than a couple of fixing dates, a PDE approach would require additional state variables or a backward recursion structure, which rapidly becomes computationally infeasible. Thus FD is impractical for 10 or more fixings.

Monte Carlo method

Monte Carlo scales naturally with the number of sampling times. Increasing the number of sample points increases only the path simulation cost, not the dimensionality of the estimation problem. Therefore, MC remains feasible and is the standard approach for discretely sampled Asian options with many observation dates.

Conclusion: Monte Carlo is the only method that remains broadly applicable and scalable in both scenarios.